In [45]:
import requests
from bs4 import BeautifulSoup
import os
import json
from tqdm import tqdm

class Thinkgood:
    def __init__(self):
        self.base_url = 'https://www.thinkcontest.com/Contest'
        self.categories = ['/CateField.html?c=11', '/CateField.html?c=12']
        self.categories_state = ['&s=hurry', '&s=ing', '&s=soon']
        self.select = 'div#wrapper > div#trunk > div#main > div.container >' \
                      'div.body.contest-cate > div.all-contest > table.type-2.mg-t-5.contest-table >' \
                      'tbody > tr > td.txt-left > div.contest-title > a '
        self.contests = dict()

    def crawling(self):
        """ 카테고리별 공모전 리스트 크롤링 """
        print("===== [Thinkgood]  Start Crawling data... =====")
        for category in self.categories:
            for state in tqdm(self.categories_state):
                page = 1
                while 42:
                    req = requests.get(self.base_url + category + state + '&page=' + str(page))
                    soup = BeautifulSoup(req.content, "html.parser")
                    data_list = soup.select(self.select)
                    self.scraping(data_list)
                    if len(data_list) == 0: 
                        break
                    page += 1
        print("===== [Thinkgood] Finish Crawling data... =====")

    def scraping(self, data_list):
        """
        공모전 세부 정보 크롤링 & dict 형태로 데이터 저장
        공모전 이름(title): [기간(host),
                          분류(classify) - 과학/공학, 소프트웨어 항목만,
                          주최자(host),
                          사이트링크(link)]
        """
        for data in data_list:
            req = requests.get('https://www.thinkcontest.com' + data.get('href'))
            soup = BeautifulSoup(req.content, "html.parser")
            tmp = soup.find(class_='body contest-detail')
            val_tmp = tmp.select('div.contest-overview > table.type-5 > tbody > tr > td')
            title = soup.find(class_='body contest-detail').find(class_='title').get_text()
            term = val_tmp[-3].get_text()
            classify = tmp.find(class_='divided').get_text().replace("\n", ", ")[2: -2]
            host = tmp.find(class_="linker").get_text().replace('\ue89e', '')
            link = tmp.find(class_="linker").get('href')
            self.contests[title] = [term, classify, host, link]

    def save(self):
#        base_dir = os.path.dirname(os.path.abspath(__file__))
#        with open(os.path.join(base_dir, 'thinkgood.json'), 'w+', encoding='utf-8') as json_file:
#            json.dump(self.contests, json_file, ensure_ascii=False, indent='\t')
        file_path = './thinkgood.json'
        with open(file_path, 'w', -1, "utf-8") as json_file:
            json.dump(self.contests, json_file, ensure_ascii=False, indent='\t')
        print("===== [Thinkgood] Save data... =====\n")

    def check_result(self):
        for key, value in self.contests.items():
            print(key, ":", value)

In [46]:
TG = Thinkgood()
TG.crawling()

  0%|                                                                                            | 0/3 [00:00<?, ?it/s]

===== [Thinkgood]  Start Crawling data... =====


100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [01:59<00:00, 39.69s/it]

===== [Thinkgood] Finish Crawling data... =====


In [47]:
len(TG.contests)

61

In [48]:
TG.save()

===== [Thinkgood] Save data... =====



In [49]:
TG.check_result()

환경 콘텐츠 비즈니스 지원사업 해커톤 모집 : ['300만원', '기획/아이디어, 과학/공학', '홈페이지', 'https://www.gcon.or.kr/busiNotice/view?pageNum=1&rowCnt=10&linkId=9967&menuId=MENU02369&schType=0&schText=&boardStyle=&categoryId=&continent=&country=']
2020년 ICT 솔루션을 활용하여 국민 생활문제를 해결하는 '솔.직 챌린지' (총 상금 2억 3천만원) : ['2020-06-01 ~ 2020-08-14', '기획/아이디어, 과학/공학, 게임/소프트웨어, 취업/창업', '솔직챌린지 접수페이지', 'http://www.ictchallenge.kr']
2020년 국민행복 IT 경진대회 : ['150만원', '과학/공학', '국민행복 IT 경진대회', 'http://find.nia.or.kr//']
(경진대회) [하포리서치코리아] Challenge 2020 (~8/19) 예측모델개발 대회 : ['2020-07-06 ~ 2020-08-19', '과학/공학, 대외활동, 해외', '홈페이지', 'http://www.haafor.com/']
시화 공상과학 프로젝트 : ['\n\n체험ㆍ참여\n\n', '과학/공학', '[2020 시화 공상(공구상가)+과학 프로젝트] 모집', 'https://form.office.naver.com/form/responseView.cmd?formkey=YThjNGY2YWEtNTcyMi00MzAxLWEzODgtOTY1MTI2N2Q0M2M4&sourceId=urlshare']
(서포터즈) 어플라이드 머티어리얼즈 코리아 서포터즈 '리얼즈' 모집 : ['체험/참여', '과학/공학, 대외활동', '어플라이드 머티어리얼즈 코리아', 'http://www.appliedmaterials.com/ko']
서울시와 ICT콤플렉스가 함께하는 2020 ICT 콕 AI 공모전 : ['2020-07-29 ~ 2020-08-12', '기획